In [21]:
%pip install azure-ai-formrecognizer
final_result=""
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = "INSERT_END_POINT_HERE!"
key = "INSERT_KEY_HERE!"

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

def analyze_read():
    formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/read.png"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    final_result=""

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-read", formUrl)
    result = poller.result()

    print ("Document contains content: ", result.content)
    return result.content





final_result=analyze_read()

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Document contains content:  While healthcare is still in the early stages of its Al journey, we are seeing pharmaceutical and other life sciences organizations making major investments in Al and related technologies." TOM LAWRY | National Director for Al, Health and Life Sciences | Microsoft
As pharmaceutical and other life sciences organizations invest in and deploy advanced technologies, they are beginning to see benefits in diverse areas across their organizations. Companies are looking to incorporate automation and continuing smart factory investments to reduce costs in drug discovery, research and development, and manufacturing and supply chain management. Many life sciences organizations are also choosing to st

In [22]:
print(final_result)

While healthcare is still in the early stages of its Al journey, we are seeing pharmaceutical and other life sciences organizations making major investments in Al and related technologies." TOM LAWRY | National Director for Al, Health and Life Sciences | Microsoft
As pharmaceutical and other life sciences organizations invest in and deploy advanced technologies, they are beginning to see benefits in diverse areas across their organizations. Companies are looking to incorporate automation and continuing smart factory investments to reduce costs in drug discovery, research and development, and manufacturing and supply chain management. Many life sciences organizations are also choosing to stay with more virtual approaches in the "new normal" - particularly in clinical trials and sales and marketing areas.
Enhancing the patient and provider experience
Clinical trial sponsors are continually seeking to make clinical trials faster and to improve the experience for patients and physicians. T

In [23]:
%pip show azure-search-documents
%pip install azure-search-documents --pre
%pip show azure-search-documents


import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

# Set the service endpoint and API key from the environment
service_name = "cogsearchdrlee"
admin_key ="1QwByQrtPi9iyANoLalq27eQhLiFa6edQ04a6iXnkdAzSeAurKLb"

index_name = "documents"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))
# Delete the index if it exists
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)


Name: azure-search-documents
Version: 11.4.0b10
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: /Users/jnoel2/Library/Python/3.9/lib/python/site-packages
Requires: azure-core, isodate, azure-common
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Name: azure-search-documents
Version: 11.4.0b10
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsof

In [24]:
# Specify the index schema
name = index_name
fields = [
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="DocumentText", type=SearchFieldDataType.String, sortable=True),

    ]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['DocumentText']}]

In [25]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    suggesters = suggester,
    cors_options=cors_options)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

Index documents created


In [26]:
documents = [
    {
    "@search.action": "upload",
    "Id": "1",
    "DocumentText": final_result
    }
]

print(documents)

[{'@search.action': 'upload', 'Id': '1', 'DocumentText': 'While healthcare is still in the early stages of its Al journey, we are seeing pharmaceutical and other life sciences organizations making major investments in Al and related technologies." TOM LAWRY | National Director for Al, Health and Life Sciences | Microsoft\nAs pharmaceutical and other life sciences organizations invest in and deploy advanced technologies, they are beginning to see benefits in diverse areas across their organizations. Companies are looking to incorporate automation and continuing smart factory investments to reduce costs in drug discovery, research and development, and manufacturing and supply chain management. Many life sciences organizations are also choosing to stay with more virtual approaches in the "new normal" - particularly in clinical trials and sales and marketing areas.\nEnhancing the patient and provider experience\nClinical trial sponsors are continually seeking to make clinical trials faster

In [27]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [28]:
question ="While healthcare is still in the early stages of its Al journey,are we seeing pharmaceutical and other life sciences organizations making major investments in Al and related technologies?"

In [29]:
import json
results =  search_client.search(search_text=question, include_total_count=True, select='Id,DocumentText')
json_results=""
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    #print("{}: {}: {}".format(result["HotelId"], result["HotelName"], result["Tags"],results["Address"]))
    json_results+=str(result)



print(json_results)

Total Documents Matching Query: 0



In [30]:
%pip install openai

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [31]:
###
import os

import requests

import json

import openai


os.environ["AZURE_OPENAI_KEY"] = "8da2d076bbce4f4995319c032629efd0"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ttai2335.openai.azure.com/"

openai.api_key = os.getenv("AZURE_OPENAI_KEY")

openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

openai.api_type = 'azure'

openai.api_version = '2023-05-15'

# Look in Azure OpenAI Studio > Deployments
deployment_name = 'gpt-35-turbo'
###

# defining a function to create the prompt from the system message and the messages
def create_prompt(system_message, messages):
    prompt = system_message
    message_template = "\n<|im_start|>{}\n{}\n<|im_end|>"
    for message in messages:
        prompt += message_template.format(message['sender'], message['text'])
    prompt += "\n<|im_start|>assistant\n"
    return prompt

# defining the system message
system_message_template = "<|im_start|>system\n{}\n<|im_end|>"
#system_message = system_message_template.format("You are an angry, you dont have a lot of patience, and you like to use curse words and you have a texan accent")
system_message = system_message_template.format("You are a scientist, expert in AI and Healthcare")
#system_message = system_message_template.format("")



In [32]:
# creating a list of messages to track the conversation
messages = [{"sender": "user", "text": "Hello, take into account the following information "+json_results},
            {"sender": "user", "text": question},

            ]
response = openai.Completion.create(
  engine=deployment_name,
  prompt= create_prompt(system_message, messages),
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
    stop=["<|im_end|>"])




print(response)

{
  "id": "cmpl-88ZaW0aaqyDkJbqhaKgPZYI242VJE",
  "object": "text_completion",
  "created": 1697054116,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": "Yes, we are seeing significant investments in AI and related technologies by pharmaceutical and other life sciences organizations. These investments are aimed at improving various aspects of healthcare, including drug discovery, clinical trial design and patient care.\n\nAI-based drug discovery is one area where significant progress is being made. Pharmaceutical companies are using AI to analyze large volumes of data and identify new drug targets and potential drug candidates. AI is also being used to design clinical trials that are more efficient, cost-effective, and more likely to yield positive results.\n\nIn patient care, AI is being used to develop personalized treatment plans based on a patient's medical history, genetic makeup, and other factors. AI is also being used to monitor patients in real-time, predict healt

In [33]:
question ="How are companies working to enchance the patient and provider experience?"

In [34]:
# creating a list of messages to track the conversation
messages = [{"sender": "user", "text": "Hello, take into account the following information "+json_results},
            {"sender": "user", "text": question},

            ]
response = openai.Completion.create(
  engine=deployment_name,
  prompt= create_prompt(system_message, messages),
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
    stop=["<|im_end|>"])




print(response)


{
  "id": "cmpl-88ZabLd7bIUPz6s7fn3apkAAL6q3x",
  "object": "text_completion",
  "created": 1697054121,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": "Companies are working to enhance the patient and provider experience in several ways using AI and other technological tools. One way is by developing virtual assistants or chatbots that can help patients schedule appointments, get information about their health conditions, and provide guidance on self-care. These virtual assistants can also help healthcare providers by automating administrative tasks such as updating patient records and sending reminders to patients.\n\nAnother way companies are enhancing the patient and provider experience is by using predictive analytics to identify patients at risk of certain health conditions or diseases. This information can help healthcare providers take proactive measures to prevent or manage these conditions, improving patient outcomes.\n\nCompanies are also developing AI-powered 

In [35]:
question ="Is healthcare is still in the early stages of its Al journey?"


In [36]:
# creating a list of messages to track the conversation
print(json_results)
messages = [{"sender": "user", "text": "Hello, take into account the following information from your database "+json_results},
            {"sender": "user", "text": question},

            ]
response = openai.Completion.create(
  engine=deployment_name,
  prompt= create_prompt(system_message, messages),
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
    stop=["<|im_end|>"])




print(response)



{
  "id": "cmpl-88ZahuU1fQ2qTuFTc1h2WWWf2ctuj",
  "object": "text_completion",
  "created": 1697054127,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": "Although there have been significant advancements in the use of AI in healthcare, it is still considered to be in the early stages of its AI journey. The healthcare industry has only begun to scratch the surface of the potential benefits that AI can offer. However, as more data becomes available and AI algorithms become more sophisticated, the potential for AI to revolutionize healthcare is tremendous.",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 73,
    "prompt_tokens": 54,
    "total_tokens": 127
  }
}
